<a href="https://colab.research.google.com/github/mkielo3/inekf_demo/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Library for Lie Groups
!git clone -q https://github.com/decargroup/pymlg.git
!pip install -q -e ./pymlg

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for pymlg (pyproject.toml) ... done
